# 第3章: 正規表現

## 20. JSONデータの読み込み
### Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [1]:
import json
import gzip
import pandas as pd

def read_gz_json(file_name):
    with gzip.open(file_name, 'r') as f:
        data = [json.loads(line) for line in f]
    return pd.DataFrame(data)
read_gz_json('data/jawiki-country.json.gz')

,title,text
0,エジプト,{{otheruses|主に現代のエジプト・アラブ共和国|古代|古代エジプト}}\n{{基礎...
1,オーストリア,{{基礎情報 国\n|略名 = オーストリア\n|日本語国名 = オーストリア共和国\n|公...
2,インドネシア,{{基礎情報 国\n| 略名 =インドネシア\n| 日本語国名 =インドネシア共和国\n| ...
3,イラク,{{複数の問題\n| 参照方法 = 2011年8月\n| 独自研究 = 2012年10月\n...
4,イラン,{{半保護}}\n{{未検証|date=2010年3月}}\n{{基礎情報 国\n | 略名...
...,...,...
243,スヴァールバル諸島およびヤンマイエン島,'''スヴァールバル諸島およびヤンマイエン島''' (Svalbard and Jan Ma...
244,シンガポール,{{参照方法|date=2012年2月}}\n{{基礎情報 国\n|略名 = シンガポール\...
245,マレーシア,{{基礎情報 国\n|略名 =マレーシア\n|日本語国名 =マレーシア\n|公式国名 ={{...
246,BES諸島,#転送 [[ボネール、シント・ユースタティウスおよびサバ]]


## 21. カテゴリ名を含む行を抽出
### 記事中でカテゴリ名を宣言している行を抽出せよ．



In [2]:
data = read_gz_json('data/jawiki-country.json.gz')
data[data.text.str.contains("Category:")]

,title,text
0,エジプト,{{otheruses|主に現代のエジプト・アラブ共和国|古代|古代エジプト}}\n{{基礎...
1,オーストリア,{{基礎情報 国\n|略名 = オーストリア\n|日本語国名 = オーストリア共和国\n|公...
2,インドネシア,{{基礎情報 国\n| 略名 =インドネシア\n| 日本語国名 =インドネシア共和国\n| ...
3,イラク,{{複数の問題\n| 参照方法 = 2011年8月\n| 独自研究 = 2012年10月\n...
4,イラン,{{半保護}}\n{{未検証|date=2010年3月}}\n{{基礎情報 国\n | 略名...
...,...,...
241,セントヘレナ・アセンションおよびトリスタンダクーニャ,{{ Infobox British territory\n| name_japanese ...
242,台湾省 (中華人民共和国),{{Otheruses|中華人民共和国の台湾省|中華民国がかつて設置していた台湾省|台湾省}...
243,スヴァールバル諸島およびヤンマイエン島,'''スヴァールバル諸島およびヤンマイエン島''' (Svalbard and Jan Ma...
244,シンガポール,{{参照方法|date=2012年2月}}\n{{基礎情報 国\n|略名 = シンガポール\...


## 22. カテゴリ名の抽出
### 記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [3]:
data.text.str.extractall('Category:(.+)]]')

0
    match                       
0   0                     エジプト|*
    1                        共和国
    2                       軍事政権
    3                フランコフォニー加盟国
1   0      オーストリアの作曲家|オーストリアの作曲家
...                          ...
244 6                  イギリス連邦加盟国
245 0                    マレーシア|*
    1                  イギリス連邦加盟国
    2                    現存する君主国
    3                      連邦制国家

[1038 rows x 1 columns]

## 23. セクション構造Permalink
### 記事中に含まれるセクション名とそのレベル（例えば”== セクション名 ==”なら1）を表示せよ．

In [4]:
result = pd.DataFrame()
for class_num in range(1,10):
    tag = "".join(["=" for _x in range(class_num + 1)])
    df = data.text.str.extractall(f'{tag} (.+) {tag}')
    if df.empty:
        break
    df["class"] = class_num
    result = pd.concat([result,df])
result

0  class
    match                    
0   0               国号      1
    1               歴史      1
    2           古代エジプト      1
    3       アケメネス朝ペルシア      1
    4          ヘレニズム文化      1
...                ...    ...
245 4             民族対立      3
29  0           サドーザイ朝      4
156 0               屋台      4
    1           フードコート      4
245 0      プラナカン（海峡華人）      4

[13058 rows x 2 columns]

## 24. ファイル参照の抽出
### 記事から参照されているメディアファイルをすべて抜き出せ．

In [5]:
data.text.str.extractall('ファイル:(.+?)\|')

0
    match                                                   
0   0                              Coat_of_arms_of_Egypt.svg
    1                                 All Gizah Pyramids.jpg
    2          Egyptiska hieroglyfer, Nordisk familjebok.png
    3      ModernEgypt, Muhammad Ali by Auguste Couder, B...
    4                                       Gamal Nasser.jpg
...                                                      ...
244 4                                         Psa keppel.JPG
    5                                           ERPBugis.JPG
    6                                Kawasaki c751 eunos.jpg
    7                                Singapore MRT Fines.jpg
245 0                           Coat of arms of Malaysia.svg

[2614 rows x 1 columns]

## 25. テンプレートの抽出Permalink
### 記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [6]:
import re
result = {}
for _index,row in data.iterrows():
    title = row.title
    text = row.text
    dic = {}
    info = False
    for sentense in text.split("\n"):
        if info:
            find_list = re.findall('\|(.+?)\s*=\s*(.+)',sentense)
            if len(find_list) == 0:
                break
            for tup in find_list:
                dic[tup[0]] = tup[1]
        if re.search("{{基礎情報",sentense):
            info = True
    result[title] = dic
result

{'エジプト': {'略名': 'エジプト',
  '漢字書き': '埃及',
  '日本語国名': 'エジプト・アラブ共和国',
  '公式国名': "{{lang|ar|'''جمهورية مصر العربية'''}}",
  '国旗画像': 'Flag of Egypt.svg',
  '国章画像': '[[ファイル:Coat_of_arms_of_Egypt.svg|100px|エジプトの国章]]',
  '国章リンク': '（[[エジプトの国章|国章]]）',
  '標語': 'なし',
  '位置画像': 'Egypt (orthographic projection).svg',
  '公用語': '[[アラビア語]]',
  '首都': '[[File:Flag of Cairo.svg|24px]] [[カイロ]]',
  '最大都市': 'カイロ',
  '元首等肩書': '[[近代エジプトの国家元首の一覧|大統領]]',
  '元首等氏名': '[[アブドルファッターフ・アッ＝シーシー]]',
  '首相等肩書': '{{ill2|エジプトの首相|en|Prime Minister of Egypt|label=首相}}',
  '首相等氏名': '{{仮リンク|ムスタファ・マドブーリー|ar|مصطفى مدبولي|en|Moustafa Madbouly}}',
  '面積順位': '29',
  '面積大きさ': '1 E12',
  '面積値': '1,010,408',
  '水面積率': '0.6%',
  '人口統計年': '2012'},
 'オーストリア': {'略名': 'オーストリア',
  '日本語国名': 'オーストリア共和国',
  '公式国名': "{{Lang|de|'''Republik Österreich'''}}",
  '国旗画像': 'Flag of Austria.svg',
  '国章画像': '[[File:Austria Bundesadler.svg|100px|オーストリアの国章]]',
  '国章リンク': '（[[オーストリアの国章|国章]]）',
  '標語': ' ',
  '国歌': "[[山岳の国、大河の国|{{lang|de|Land der Berge, Land 

## 26. 強調マークアップの除去
### 25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: マークアップ早見表）．

In [9]:
def delete_emphasis(dic:dict) -> dict:
    for country in dic.keys():
        for key in dic[country].keys():
            dic[country][key] = dic[country][key].replace("'''","")
    return dic
result = delete_emphasis(result)
result

## 27. 内部リンクの除去
### 26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: マークアップ早見表）．



In [10]:
def check_inner(text:str) -> bool:
    file_search = re.search("ファイル:(.+?)\|",text)
    if file_search is not None:
        return False
    file_search = re.search("File:(.+?)\|",text)
    if file_search is not None:
        return False
    return True


def delete_innerlink(dic:dict) -> dict:
    for country in dic.keys():
        for key in dic[country].keys():
            inner_link = re.findall("\[\[(.+?)\]\]",dic[country][key])
            if len(inner_link) == 0:
                continue
            new_text = []
            for item in inner_link:
                if not check_inner(item):
                    new_text.append(f"[[{item}]]")
                else:
                    new_text.append(item.split("|")[-1])
            dic[country][key] = ",".join(new_text)
    return dic
result = delete_innerlink(result)
result

## 28. MediaWikiマークアップの除去Permalink
### 27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

In [11]:
def delete_markup(dic:dict) -> dict:
    for country in dic.keys():
        for key in dic[country].keys():
            dic[country][key] = re.sub("\{\{(.+?)\}\}","\\1",dic[country][key])        
    return dic
result = delete_markup(result)
result

## 29. 国旗画像のURLを取得する
### テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: MediaWiki APIのimageinfoを呼び出して，ファイル参照をURLに変換すればよい）

In [12]:
import json
import requests
def get_flagimage(country_dict:dict)->dict:
    if '国旗画像' not in country_dict:
        return {}
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "prop": "imageinfo",
        "titles": f"File:{country_dict['国旗画像']}",
        "iiprop":"url"
    }
    r = requests.get(url=url, params=params)
    request_result = json.loads(r.text)
    image_dict = {}
    for page in request_result['query']['pages'].values():
        image_dict[page["title"]] = [image['url'] for image in page['imageinfo']]
    return image_dict

flag_dict = {}
for country in result.keys():
    flag_dict[country] = get_flagimage(result[country])
flag_dict

{'エジプト': {'File:Flag of Egypt.svg': ['https://upload.wikimedia.org/wikipedia/commons/f/fe/Flag_of_Egypt.svg']},
 'オーストリア': {'File:Flag of Austria.svg': ['https://upload.wikimedia.org/wikipedia/commons/4/41/Flag_of_Austria.svg']},
 'インドネシア': {},
 'イラク': {'File:Flag of Iraq.svg': ['https://upload.wikimedia.org/wikipedia/commons/f/f6/Flag_of_Iraq.svg']},
 'イラン': {},
 'チュニジア': {},
 'トルコ': {'File:Flag of Turkey.svg': ['https://upload.wikimedia.org/wikipedia/commons/b/b4/Flag_of_Turkey.svg']},
 'ロシア': {'File:Flag of Russia.svg': ['https://upload.wikimedia.org/wikipedia/en/f/f3/Flag_of_Russia.svg']},
 'コスタリカ': {},
 'ラオス': {},
 '中華人民共和国': {"File:Flag of the People's Republic of China.svg": ['https://upload.wikimedia.org/wikipedia/commons/f/fa/Flag_of_the_People%27s_Republic_of_China.svg']},
 'インド': {'File:Flag of India.svg': ['https://upload.wikimedia.org/wikipedia/en/4/41/Flag_of_India.svg']},
 'パラグアイ': {},
 'アルゼンチン': {},
 'ウルグアイ': {},
 'チリ': {'File:Flag of Chile.svg': ['https://upload.wikime